In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os
import mne
import torch.nn.functional as F
import datetime
import pandas as pd
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import torch
import pywt
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
def df_from_csv(file_path):
    # Read the csv file
    df_in = pd.read_csv(file_path)

    # Get the header row
    header_row = list(df_in.columns)

    # Convert the header row to integer values
    header_row = list(map(int, header_row))

    # Create a DataFrame from the header row
    header_df = pd.DataFrame([header_row], columns=df_in.columns)

    # Concatenate the header row with the original DataFrame
    df_imm = pd.concat([header_df, df_in], ignore_index=True)

    # Reset column names
    df_imm.columns = range(df_imm.shape[1])

    # Remove the first column
    df_imm = df_imm.drop(0, axis=1)

    df_out = df_imm.T
    
    return df_out

In [1]:
def z_score(epoch):    
    # Apply z-score normalization to each channel, saved in epoch
    for i in range(epoch.shape[0]):
        channel_epoch = epoch.iloc[i]
        mean = np.mean(channel_epoch)
        std = np.std(channel_epoch)
        z_scored_epoch = (channel_epoch - mean) / std
        epoch.iloc[i] = z_scored_epoch
    
    return epoch

def frequency_to_scale(freq, wavelet='morl', sampling_rate=250):
    # For the Morlet wavelet, scales are inversely proportional to frequency
    center_freq = pywt.central_frequency(wavelet)
    return center_freq / (freq / sampling_rate)

def apply_wavelet_transform(data_norm, wavelet='morl', freq_range=(8, 30), sampling_rate=250):
    """
    Apply wavelet transform to EEG data.
    
    Parameters:
    data_norm (ndarray): 2D array with shape (channels, time_points)
    wavelet (str): Wavelet type (default 'morl')
    freq_range (tuple): Frequency range for the CWT (default (8, 30) Hz)
    sampling_rate (int): Sampling rate of the EEG data (default 250 Hz)
    
    Returns:
    ndarray: 3D array with shape (channels, scales, time_points)
    """
    n_channels, n_times = data_norm.shape
    # Define scales based on the desired frequency range
    scales = frequency_to_scale(np.arange(freq_range[0], freq_range[1]+1), wavelet=wavelet, sampling_rate=sampling_rate)
    
    coeffs = []
    for i in range(n_channels):
        # Compute the wavelet transform coefficients
        coef, _ = pywt.cwt(data_norm.iloc[i], scales=scales, wavelet=wavelet)
        coeffs.append(coef)
    
    # Stack coefficients to form a 3D tensor
    coeffs_done = np.stack(coeffs, axis=0)
    
    return coeffs_done

def apply_bandpass_filter(signal, b, a):
    return filtfilt(b, a, signal)

def filtering_raw_signal(file_path):
    # Load the data, in dataframe format
    data_raw = df_from_csv(file_path)
    
    # Filter parameters
    l_freq = 8
    h_freq = 30
    order = 5
    fs = 250

    nyquist = 0.5 * fs
    low = l_freq / nyquist
    high = h_freq / nyquist

    # Calculate filter coefficients
    b, a = butter(order, [low, high], btype='band')
    
    # Apply the filter to each row independently
    for i in range(data_raw.shape[0]):
        data_raw.iloc[i] = apply_bandpass_filter(data_raw.iloc[i], b, a)
        
    # Convert filtered data back to DataFrame
    data_filtered = pd.DataFrame(data_raw)
    
    # Split data into segments of 0.5 seconds, for epoching
    epoch_size = int(fs * 0.5)  # Number of samples in 0.5 seconds
    epochs = [data_filtered.iloc[:, i:i+epoch_size] for i in range(0, len(data_filtered.columns), epoch_size)]
    
    # Create an empty list to store the transformed epochs
    transformed_epochs = []
    
    # Z-score normalization and wavelet transform for each segment and stack them into a tensor (4D array), 
    # Loop through each epoch
    for epoch in epochs:
        # Z-score each epoch
        epoch_norm = z_score(epoch)
        
        # # Apply wavelet transformation
        wavelet_tensor = apply_wavelet_transform(epoch_norm)
        
        # Append the transformed epoch to the list
        transformed_epochs.append(wavelet_tensor)
        
        
    # Convert the list of epochs into a tensor dataset
    tensor_dataset = torch.tensor(transformed_epochs, dtype=torch.float32)  # Ensure the tensor is float32
     
    return tensor_dataset

In [8]:
a = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\vue\lefts_adnane.csv"
left_tensor = filtering_raw_signal(a)

b = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\vue\rights_adnane.csv"
right_tensor = filtering_raw_signal(b)

# Array of 50 zeros
zeross = np.zeros(50)

# Array of 50 ones
oness = np.ones(50)

C:\Users\chakr\AppData\Local\Temp\ipykernel_15544\858416405.py:93: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  tensor_dataset = torch.tensor(transformed_epochs, dtype=torch.float32)  # Ensure the tensor is float32


In [29]:
import torch
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader

# Assuming you have two tensors: tensor1 and tensor2, and their corresponding label arrays: labels1 and labels2
tensor1 = left_tensor  # Your first tensor
tensor2 = right_tensor  # Your second tensor
labels1 = zeross  # Labels corresponding to tensor1
labels2 = oness  # Labels corresponding to tensor2

# Convert label arrays to tensors
labels1_tensor = torch.tensor(labels1)
labels2_tensor = torch.tensor(labels2)

# In long format
labels1_tensor = torch.tensor(labels1, dtype=torch.float32).to(torch.long)
labels2_tensor = torch.tensor(labels2, dtype=torch.float32).to(torch.long)
labels = torch.cat((labels1_tensor, labels2_tensor))

# Create datasets from tensors and labels
dataset1 = TensorDataset(tensor1, labels1_tensor)
dataset2 = TensorDataset(tensor2, labels2_tensor)

# Concatenate the datasets
combined_dataset = ConcatDataset([dataset1, dataset2])

train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_dataset, val_dataset = random_split(combined_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [19]:
class Advanced_EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN, self).__init__()
        # First convolutional layer with Batch Normalization and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Second convolutional layer with Batch Normalization and ReLU activation
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Third convolutional layer with Batch Normalization and ReLU activation
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 2 * 15, 256)  # Adjust based on final output size after conv layers
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        # Forward pass through the first convolutional layer
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Forward pass through the second convolutional layer
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Forward pass through the third convolutional layer
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self._forward_conv_layers(x)
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        # Forward pass through the first fully connected layer with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Forward pass through the second fully connected layer with ReLU
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x

In [20]:
class Advanced_EEG_MI_CNN_LSTM_Transformer(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN_LSTM_Transformer, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=128*2*15, hidden_size=128, num_layers=1, batch_first=True)
        
        # Transformer layer
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def _forward_conv_layers(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        batch_size = x.size(0)
        # Forward pass through convolutional layers
        x = self._forward_conv_layers(x)
        
        # Flatten for LSTM
        x = x.view(batch_size, -1, 128*2*15)
        
        # Forward pass through LSTM
        x, (hn, cn) = self.lstm(x)
        
        # Transformer encoding
        x = self.transformer_encoder(x)
        
        # Take the output from the last sequence step for classification
        x = x[:, -1, :]
        
        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [30]:
# Save the model with date and time in the filename
def save_model(model, path):
    torch.save(model.state_dict(), path)

# Example usage:
model = Advanced_EEG_MI_CNN_LSTM_Transformer()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model with current date and time
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_dir = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\models"
print("Model directory:", model_dir)
os.makedirs(model_dir, exist_ok=True)
model_filename = os.path.join(model_dir, f'{model.__class__.__name__}_{current_time}.pth')
print("Model filename:", model_filename)
save_model(model, model_filename)
print(f'Model saved as {model_filename}')

Epoch 1/50, Loss: 0.7346
Validation Loss: 0.6893, Validation Accuracy: 0.5500
Epoch 2/50, Loss: 0.6907
Validation Loss: 0.6926, Validation Accuracy: 0.5000
Epoch 3/50, Loss: 0.6817
Validation Loss: 0.6813, Validation Accuracy: 0.5500
Epoch 4/50, Loss: 0.5504
Validation Loss: 0.6468, Validation Accuracy: 0.7000
Epoch 5/50, Loss: 0.3380
Validation Loss: 0.9555, Validation Accuracy: 0.6000
Epoch 6/50, Loss: 0.0833
Validation Loss: 1.0555, Validation Accuracy: 0.7500
Epoch 7/50, Loss: 0.5971
Validation Loss: 0.9994, Validation Accuracy: 0.7000
Epoch 8/50, Loss: 0.3674
Validation Loss: 0.6293, Validation Accuracy: 0.6500
Epoch 9/50, Loss: 0.4005
Validation Loss: 0.8803, Validation Accuracy: 0.7500
Epoch 10/50, Loss: 0.1678
Validation Loss: 0.7189, Validation Accuracy: 0.8000
Epoch 11/50, Loss: 0.1201
Validation Loss: 1.0590, Validation Accuracy: 0.7500
Epoch 12/50, Loss: 0.0928
Validation Loss: 0.9076, Validation Accuracy: 0.7000
Epoch 13/50, Loss: 0.2321
Validation Loss: 1.0860, Validation